In [ ]:
import os
import re

import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

import gvar as gv

import seaborn as sns
import matplotlib.pylab as plt

from luescher_nd.database import utilities as ut
from luescher_nd.database.utilities import DATA_FOLDER
from luescher_nd.zeta.extern.pyzeta import zeta
from luescher_nd.zeta.extern.pyzeta import zeta as spherical_zeta

from luescher_nd.plotting import styles

from luescher_nd.database.utilities import get_continuum_extrapolation

from luescher_nd.hamiltonians.kinetic import MomentumKineticHamiltonian
from luescher_nd.solvers.contact import FitKernel

from luescher_nd.database.utilities import get_degeneracy

from luescher_nd.plotting.styles import MARKERS
from luescher_nd.database.utilities import get_degeneracy

from luescher_nd.database.utilities import _poly as polyfcn

import matplotlib

styles.setup(pgf=False)

In [ ]:
A1_WIDTH = 6.0

In [ ]:
matplotlib.use("pgf")
sns.set(
    context="paper",
    style="ticks",
    font_scale=1/1.7,
    rc={
        # "mathtext.fontset": "cm",
        "pgf.rcfonts": False,
        "axes.unicode_minus": False,
        "font.serif": [],
        "font.sans-serif": [],
    },
)

In [ ]:
%load_ext blackcellmagic

In [ ]:
a_inv = 0.0
L = 1.0

In [ ]:
dummy = MomentumKineticHamiltonian(1)
kernel = FitKernel(dummy, spherical_zeta, 0.0)

In [ ]:
poles = list(get_degeneracy(10).keys())
pole_map = {}

zeros = {}
n = 0
nn = 0
start = -20
for pole, degs in get_degeneracy(10).items():
    end = pole
    
    for deg in degs:
        
        zero = kernel.get_zeta_intersection(
            bounds=(start + 1.0e-2, end - 1.0e-2),
            method="bounded",
            options={"xatol": 1.0e-16},
        )
        zeros[nn] = zero
        pole_map[nn] = (start, end)
        nn += 1
    
    start = end
    n += 1

In [ ]:
files = [f for f in os.listdir(DATA_FOLDER) if f.endswith(".sqlite") and not "tmp" in f]
files

In [ ]:
file_name = f"contact-fitted_a-inv={a_inv:+1.1f}_zeta=spherical_projector=a1g_n-eigs=200.sqlite"
print(file_name)
df = ut.read_table(
    os.path.join(DATA_FOLDER, file_name),
    zeta=None,
    round_digits=2,
    filter_poles=False,
    filter_by_nstates=False,
    filter_degeneracy=False,
).query("nlevel < 24 and n1d > 5")[
    ["n1d", "epsilon", "nstep", "L", "x", "nlevel", "mass"]
]
df["L"] = df.L.round(7)
df.head()

def nstep_label(nstep) -> str:
    return "$" + (str(nstep) if nstep > 0 else r"\infty") + "$"

df["nstep_label"] = df.nstep.apply(nstep_label)

In [ ]:
fit = get_continuum_extrapolation(df, n_poly_max=6, odd_poly=True).query("n_poly_max > 1")

In [ ]:
def avg_gvars(ys):
    y_mean = y_sdev = 0
    len_ys = len(ys) 
        
    for y in ys:
        y_mean += y / len_ys
    for y in ys:
        y_sdev += (y - y_mean)**2 / len_ys
        

    y_mean += gv.gvar([0]*len(y_sdev), np.sqrt(gv.mean(y_sdev)))

    return y_mean

In [ ]:
def plot_frame(*args, **kwargs):
    x_key, y_key = args
    df = kwargs.pop("data")
    eps = df[x_key].values
    x = df[y_key].values

    nstep = df.nstep.unique()[0]
    nlevel = df.nlevel.unique()[0]
    L = df.L.unique()[0]
    fits = fit.query("L == @L and nlevel == @nlevel and nstep == @nstep")

    ax = plt.gca()
    ax.plot(eps, x, **kwargs)

    ys = []
    xs = []
    eps = np.linspace(1.0e-3, eps.max())
    for idx, fit_data in fits.iterrows():
        x = []
        xs.append(np.array([fit_data["x0"]]))
        for key, val in fit_data.items():
            if re.match("x[0-9]+", key):
                if not pd.isna(val):
                    x.append(val)
        p = {"x": x}
        ys.append(polyfcn(eps, p, even=False))

    y = avg_gvars(ys)
    x = avg_gvars(xs)

    ax.plot(eps, gv.mean(y), color="green")
    ax.fill_between(
        eps,
        gv.mean(y) - gv.sdev(y),
        gv.mean(y) + gv.sdev(y),
        color="green",
        alpha=0.5,
        zorder=-3,
    )

    ax.axhline(zeros[nlevel], color="black", lw=1, ls="--", zorder=-2)
    ax.axhline(gv.mean(x), color="blue", lw=0.5, zorder=-3, alpha=0.8)
    ax.axhspan(
        gv.mean(x) - gv.sdev(x),
        gv.mean(x) + gv.sdev(x),
        color="blue",
        alpha=0.1,
        zorder=-4,
    )
    ax.set_ylim(pole_map[nlevel])
    ax.set_yticks([pole_map[nlevel][0], zeros[nlevel], pole_map[nlevel][1]])
    ax.annotate(
        str(x[0]),
        xy=(2.0e-2, gv.mean(x)),
        xytext=(3.0e-2, gv.mean(x) + max(0.3, 0.1 + gv.sdev(x))),
        color="blue",
        alpha=0.8,
        zorder=-1,
        arrowprops={"arrowstyle": "-", "color": "blue", "alpha": 0.8, "zorder": -10, "lw":0.5 },
    )


In [ ]:
eps = np.linspace(1.0e-3, df.epsilon.max(), 100)

tf = df.loc[df.nlevel % 2 == 0].query("nlevel < 20 and nlevel > 0 and L == 1").sort_values("x", ascending=True)

grid = sns.FacetGrid(
    data=tf,
    col="nstep",
    row="nlevel",
    sharex=True,
    sharey="row",
    legend_out=True,
    hue_kws={
        "marker": MARKERS * 5,
        "ms": [2] * 40,
        "lw": [0.5] * 40,
        "ls": ["None"]*40,
        "color": ["black"]*40,
        "zorder": [3]*40,
    },
    margin_titles=True,
    col_order=[nstep for nstep in [1, 2, 4, -1]],
    row_order=np.sort(tf.nlevel.unique())[::-1],
    xlim=(eps.min(), eps.max()*1.1),
    aspect=2, height=1.5
)

grid.map_dataframe(plot_frame, "epsilon", "x")

for ax in grid.axes.flat:
    ax.set_xscale("log")
    
grid.set_ylabels(r"$x = \frac{2 \mu E L^2}{4 \pi^2}$")
grid.set_xlabels(r"$\epsilon [\mathrm{fm}]$")
grid.set_titles(
    #row_template=r"${row_var} = {row_name} [\mathrm{{fm}}]$",
    col_template=r"$n_{{\mathrm{{step}}}} = {col_name}$",
)

plt.subplots_adjust(wspace=0.2, hspace=0.2)

ratio = grid.fig.get_figheight() / grid.fig.get_figwidth()
grid.fig.set_figheight(ratio * A1_WIDTH)
grid.fig.set_figwidth(A1_WIDTH)

grid.fig.text(
    0.5,
    0.5,
    "Preliminary",
    fontsize=60,
    color="grey",
    ha="center",
    va="center",
    alpha=0.2,
    rotation=30,
    zorder=10,
)



grid.fig.set_dpi(300)

In [ ]:
grid.savefig("continuum-extrapolation.pgf", bbox_axes="tight")

In [ ]:
def gvar_zeta_x(x):
    mean, sdev = gv.mean(x), gv.sdev(x)
    x_range = pd.Series(np.linspace(mean-sdev, mean+sdev, 50), name="x")
    x_range.index.name = "linspace_id"
    return x_range

def gvar_zeta(x):
    mean, sdev = gv.mean(x), gv.sdev(x)
    x_range = np.linspace(mean-sdev, mean+sdev, 50)
    y_range = []
    for xx in x_range:
        y_range.append(spherical_zeta(xx)[0])
    ddf = pd.DataFrame(np.transpose([x_range, y_range]), columns=["x", "y"])
    ddf.index.name = "linspace"
    return ddf

In [ ]:
def gv_mean(x):
    len_x = len(x) if hasattr(x, "__iter__") else 1
    mean = np.sum(x) / len_x
    sdev = gv.sqrt(np.sum((x - mean)**2 / len_x))
    return mean + gv.gvar(0, gv.mean(sdev))

In [ ]:
DEGS = get_degeneracy(20).keys()

def gv_compare(gvar, x, sigmas=1):
    m, s = gv.mean(gvar), gv.sdev(gvar)
    return m - s * sigmas < x and m + s * sigmas > x

def in_deg(gvar, degs=DEGS, sigmas=1):
    in_deg = False
    for deg in degs:
        in_deg = gv_compare(gvar, deg, sigmas=sigmas)
        if in_deg:
            break
    return in_deg

model_avg = fit.groupby(["L", "nstep", "nlevel"])[["x0"]].agg(gv_mean)
model_avg.tail()

model_avg["in_deg"] = model_avg.x0.apply(in_deg, sigmas=3)

model_avg = model_avg.query("in_deg == False").copy()

In [ ]:
zeta = model_avg.apply(lambda row: gvar_zeta_x(row["x0"]), axis=1).stack().reset_index()
zeta = zeta.rename(columns={0: "x"})
zeta["y"] = zeta.apply(lambda row: spherical_zeta(row["x"])[0]/ np.pi / row["L"], axis=1)
zeta.tail()

In [ ]:
zeta.tail()

zeta_condensed = zeta.groupby(["L", "nstep", "nlevel"])[["x", "y"]].agg(["mean", "max", "min"])
zeta_condensed.head()

In [ ]:
def plot_err(*args, **kwargs):
    tf = kwargs.pop("data")

    s = tf[("y", "max")]
    tf["err"] = s.where(s > -tf[("y", "min")], -tf[("y", "min")])
    tf["err_acc"] = tf.err.cumsum()
    for alpha, df in zip(
        [0.3, 0.1], 
        [tf.loc[tf.err_acc < 10]]#, tf.loc[tf.err_acc >= 10]]
    ):
        x_mean = df[("x", "mean")]
        x_min = df[("x", "min")]
        x_max = df[("x", "max")]
        y_mean = df[("y", "mean")]
        y_min = df[("y", "min")]
        y_max = df[("y", "max")] - y_mean
        nstep = tf.nstep.unique()[0]

        ax = plt.gca()

        ax.plot(x_mean, y_mean, **kwargs)
        ax.fill_between(
            x_mean, y_min, y_max, alpha=alpha, zorder=-1, color=kwargs.get("color"), ls="None"
        )


In [ ]:
grid = sns.FacetGrid(
    data=zeta_condensed.reset_index(),
    col="L",
    hue="nstep",
    sharex=True,
    sharey="row",
    legend_out=True,
    hue_kws={
        "marker": MARKERS * 5,
        "ms": [2] * 40,
        "lw": [0.5] * 40,
        "ls": ["--"] * 40,
        "zorder": [3] * 40,
    },
    margin_titles=True,
    hue_order=[nstep for nstep in [1, 2, 4, -1]],
    # ylim=(-10,10),
    xlim=(-0.5, 20),
)

grid.map_dataframe(plot_err)

for ax in grid.axes.flat:
    ax.axhline(0, color="black", ls="--")

grid.add_legend()

grid.set_xlabels(r"$x = \frac{2 \mu E L^2}{4 \pi^2}$")
grid.set_ylabels(r"$p \cot\delta(p) \, [\mathrm{{fm}}^{-1}]$")
grid.set_titles(
    # row_template=r"${row_var} = {row_name} [\mathrm{{fm}}]$",
    col_template=r"$L = {col_name} \, [\mathrm{{fm}}]$"
)

plt.subplots_adjust(wspace=0.2, hspace=0.2)

ratio = grid.fig.get_figheight() / grid.fig.get_figwidth()
grid.fig.set_figheight(ratio * A1_WIDTH)
grid.fig.set_figwidth(A1_WIDTH)


grid.fig.set_dpi(300)

grid.fig.text(
    0.5,
    0.5,
    "Preliminary",
    fontsize=60,
    color="grey",
    ha="center",
    va="center",
    alpha=0.2,
    rotation=30,
    zorder=10,
)

plt.show()


In [ ]:
grid.savefig("continuum-extrapolation-ere.pgf", bbox_axes="tight")